# Part 2 of "Exploring GAN latent space to generate images with desired features​ by Digital Sreeni": Arithmetic with GAN latent vectors (Train)

1) Link to the Youtube tutorial video: https://www.youtube.com/watch?v=iuQ_f3W5Ttk&list=PLZsOBAyNTZwboR4_xj-n3K6XBTweC4YVD&index=13
2) Link to the UTKFace dataset: https://susanqq.github.io/UTKFace/

In [1]:
"""
Dataset from: Dataset from: https://susanqq.github.io/UTKFace/ 

Latent space is hard to interpret unless conditioned using many classes.​
But, the latent space can be exploited using generated images.​
Here is how...

x Generate 10s of images using random latent vectors.​
x Identify many images within each category of interest (e.g., smiling man, neutral man, etc. )​
x Average the latent vectors for each category to get a mean representation in the latent space (for that category).​
x Use these mean latent vectors to generate images with features of interest. ​

This part of the code is used to train a GAN on 128x128x3 images.(e.g. Human Faces data)
The generator model can then be used to generate new images. (new faces)
The features in the new images can be 'engineered' by doing simple arithmetic
between vectors that are used to generate images. 
In summary, you can find the latent vectors for Smiling Man, neutral face man, 
and a baby with neutral face and then generate a smiling baby face by:
    Smiling Man + Neutral Man - Neutral baby = Smiling Baby
"""

# Import the required libraries
from numpy import zeros, ones
from numpy.random import randn, randint

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
from tensorflow.keras.utils import plot_model

from matplotlib import pyplot as plt


# Supplementary Part

## Define the function to create/build/define and compile the discriminator network

In [2]:
# define the standalone discriminator model
# Input would be 128x128x3 images and the output would be a binary (using sigmoid)
#Remember that the discriminator is just a binary classifier for true/fake images.
def define_discriminator(in_shape=(128,128,3)):
	model = Sequential()
	# normal
	model.add(Conv2D(128, (3,3), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 64x64
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 32x32
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 16x16
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 8x8
	model.add(Conv2D(128, (3,3), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

#Verify the model summary
test_discr = define_discriminator()
print(test_discr.summary())
plot_model(test_discr, to_file='disc_model.png', show_shapes=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 128)     3584      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 128)     0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 128)       147584    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       147584    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       1

c:\Users\weiyo\anaconda3\envs\GPUEnv\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


## Define the function to create/build/define the generator network

In [3]:
# define the standalone generator model
# Generator must generate 128x128x3 images that can be fed into the discriminator. 
# So, we start with enough nodes in the dense layer that can be gradually upscaled
#to 128x128x3. 
#Remember that the input would be a latent vector (usually size 100)
def define_generator(latent_dim):
	model = Sequential()
	# Define number of nodes that can be gradually reshaped and upscaled to 128x128x3
	n_nodes = 128 * 8 * 8 #8192 nodes
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((8, 8, 128)))
	# upsample to 16x16
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 32x32
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 64x64
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 128x128
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# output layer 128x128x3
	model.add(Conv2D(3, (8,8), activation='tanh', padding='same')) #tanh goes from [-1,1]
	return model

test_gen = define_generator(100)
print(test_gen.summary())
plot_model(test_gen, to_file='generator_model.png', show_shapes=True)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8192)              827392    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 8192)              0         
_________________________________________________________________
reshape (Reshape)            (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 16, 16, 128)       262272    
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 128)       262272    
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 32, 32, 128)      

## Define the function to create/build/define and compile the GAN model

In [4]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

test_gan = define_gan(test_gen, test_discr)
print(test_gan.summary())
plot_model(test_gan, to_file='combined_model.png', show_shapes=True)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_1 (Sequential)    (None, 128, 128, 3)       1901059   
_________________________________________________________________
sequential (Sequential)      (None, 1)                 602113    
Total params: 2,503,172
Trainable params: 1,901,059
Non-trainable params: 602,113
_________________________________________________________________
None
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


## Define a function to select a half batch size of real images (features + labels) from the dataset, then assign each selected real image a value of 1 as ground truth

In [5]:
# Function to sample some random real images
def generate_real_samples(dataset, n_samples):
	ix = randint(0, dataset.shape[0], n_samples)
	X = dataset[ix]
	y = ones((n_samples, 1)) # Class labels for real images are 1
	return X, y

## Define a function to generate half batch size of random noise/latent vector as the input of the generator network

In [6]:
# Function to generate random latent points
def generate_latent_points(latent_dim, n_samples):
	x_input = randn(latent_dim * n_samples)
	x_input = x_input.reshape(n_samples, latent_dim) #Reshape to be provided as input to the generator. 
	return x_input

## Define a function to generate images (fake images) using half batch size of random noise/latent vector

In [7]:
# Function to generate fake images using latent vectors
def generate_fake_samples(g_model, latent_dim, n_samples):
	x_input = generate_latent_points(latent_dim, n_samples) #Generate latent points as input to the generator
	X = g_model.predict(x_input) #Use the generator to generate fake images
	y = zeros((n_samples, 1)) # Class labels for fake images are 0
	return X, y

## Define a function to save the model performance for every N number of epochs

In [8]:
# Function to save Plots after every n number of epochs
def save_plot(examples, epoch, n=10):
	# The path to the cropped image folder
	path_trainingdata = "D:/AI_Master_New/Under_Local_Git_Covered/Deep_Learning_Tutorials_codebasics/Generative_Adversarial_Network_GAN/Explore_GAN_LatentSpace_Tutorial13/saved_data_during_training/images"
	# Create the directory (folder) if the directory is not exist. If the directory exists, just skip it (nothing happens).
	import os
	os.makedirs(path_trainingdata, exist_ok = True)
	# scale images from [-1,1] to [0,1] so we can plot
	examples = (examples + 1) / 2.0
	for i in range(n * n):
		plt.subplot(n, n, 1 + i)
		plt.axis('off')
		plt.imshow(examples[i])
	# save plot to a file so we can view how generated images evolved over epochs
	filename = path_trainingdata + '/' + 'generated_plot_128x128_e%03d.png' % (epoch+1)
	plt.savefig(filename)
	plt.close()

## Define a function to summarize the model performance periodically

In [9]:
# Function to summarize performance periodically. 
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
	# Fetch real images
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real images - get accuracy
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# Generate fake images
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake images - get accuracy
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# Print discriminate accuracies on ral and fake images. 
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save generated images periodically using the save_plot function
	save_plot(x_fake, epoch)
	# The path to the cropped image folder
	path_trainingmodel = "D:/AI_Master_New/Under_Local_Git_Covered/Deep_Learning_Tutorials_codebasics/Generative_Adversarial_Network_GAN/Explore_GAN_LatentSpace_Tutorial13/saved_data_during_training/models"
	# Create the directory (folder) if the directory is not exist. If the directory exists, just skip it (nothing happens).
	import os
	os.makedirs(path_trainingmodel, exist_ok = True)
	# save the generator model
	filename = path_trainingmodel + '/' + 'generator_model_128x128_%03d.h5' % (epoch+1)
	g_model.save(filename)


## Define the function to perform the GAN model training

In [10]:
# train the generator and discriminator by enumerating batches and epochs. 
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=128):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2) #Disc. trained on half batch real and half batch fake images
	#  enumerate epochs
	for i in range(n_epochs):
		# enumerate batches 
		for j in range(bat_per_epo):
			# Fetch random 'real' images
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# Train the discriminator using real images
			d_loss1, _ = d_model.train_on_batch(X_real, y_real)
			# generate 'fake' images 
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# Train the discriminator using fake images
			d_loss2, _ = d_model.train_on_batch(X_fake, y_fake)
			# Generate latent vectors as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# Label generated (fake) mages as 1 to fool the discriminator 
			y_gan = ones((n_batch, 1))
			# Train the generator (via the discriminator's error)
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# Report disc. and gen losses. 
			print('Epoch>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, bat_per_epo, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

# Main Part

## Load dataset

In [11]:
#Now that we defined all necessary functions, let us load data and train the GAN.
# Dataset from: https://susanqq.github.io/UTKFace/
import os
import numpy as np
import cv2
from PIL import Image
import random

 
""" n=20000 #Number of images to read from the directory. (For training)
SIZE = 128 #Resize images to this size
all_img_list =  os.listdir('D:/AI_Master_New/Under_Local_Git_Covered/Deep_Learning_Tutorials_codebasics/Generative_Adversarial_Network_GAN/Explore_GAN_LatentSpace_Tutorial13/data/cropped_images/') 

dataset_list = random.sample(all_img_list, n) #Get n random images from the directory
 """
# Get all images in the directory
dataset_list = os.listdir('D:/AI_Master_New/Under_Local_Git_Covered/Deep_Learning_Tutorials_codebasics/Generative_Adversarial_Network_GAN/Explore_GAN_LatentSpace_Tutorial13/data/cropped_images/') 
SIZE = 128 # Resize images to this size

#Read images, resize and capture into a numpy array
dataset = []
for img in dataset_list:
    temp_img = cv2.imread("D:/AI_Master_New/Under_Local_Git_Covered/Deep_Learning_Tutorials_codebasics/Generative_Adversarial_Network_GAN/Explore_GAN_LatentSpace_Tutorial13/data/cropped_images/" + img)
    temp_img = cv2.cvtColor(temp_img, cv2.COLOR_BGR2RGB) #opencv reads images as BGR so let us convert back to RGB
    temp_img = Image.fromarray(temp_img)
    temp_img = temp_img.resize((SIZE, SIZE)) #Resize
    dataset.append(np.array(temp_img))   

dataset = np.array(dataset) #Convert the list to numpy array

## Data preprocessing

In [12]:
#Rescale to [-1, 1] - remember that the generator uses tanh activation that goes from -1,1
dataset = dataset.astype('float32')
	# scale from [0,255] to [-1,1]
dataset = (dataset - 127.5) / 127.5

## Create generator, discriminator, and GAN models before the training

In [13]:
# size of the latent space
latent_dim = 100
# create the discriminator using our pre-defined function
d_model = define_discriminator()
# create the generator using our pre-defined function
g_model = define_generator(latent_dim)
# create the gan  using our pre-defined function
gan_model = define_gan(g_model, d_model)

## Perform GAN model training

In [14]:
# train model
train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=22, n_batch=16) # We set the batch_size=16 (extremely small) only because if go beyond this value, we will face OOM (out of memory) error on this device. The batch_size must be an even integer because we use it to calculate half_batch for discriminator. On this device, batch_size=16 for each image dimension 128x128 pixels take 16 minutes for 1 epoch training. 


Epoch>1, 1/510, d1=0.696, d2=0.695 g=0.692
Epoch>1, 2/510, d1=0.630, d2=0.695 g=0.692
Epoch>1, 3/510, d1=0.562, d2=0.697 g=0.691
Epoch>1, 4/510, d1=0.410, d2=0.704 g=0.686
Epoch>1, 5/510, d1=0.219, d2=0.725 g=0.668
Epoch>1, 6/510, d1=0.080, d2=0.787 g=0.637
Epoch>1, 7/510, d1=0.049, d2=0.877 g=0.614
Epoch>1, 8/510, d1=0.030, d2=0.899 g=0.649
Epoch>1, 9/510, d1=0.046, d2=0.783 g=0.723
Epoch>1, 10/510, d1=0.090, d2=0.675 g=0.791
Epoch>1, 11/510, d1=0.094, d2=0.655 g=0.823
Epoch>1, 12/510, d1=0.026, d2=0.634 g=0.856
Epoch>1, 13/510, d1=0.150, d2=0.631 g=0.805
Epoch>1, 14/510, d1=0.161, d2=0.686 g=0.780
Epoch>1, 15/510, d1=0.037, d2=0.635 g=0.769
Epoch>1, 16/510, d1=0.011, d2=0.631 g=0.778
Epoch>1, 17/510, d1=0.070, d2=0.624 g=0.783
Epoch>1, 18/510, d1=0.038, d2=0.618 g=0.794
Epoch>1, 19/510, d1=0.010, d2=0.598 g=0.822
Epoch>1, 20/510, d1=0.031, d2=0.574 g=0.859
Epoch>1, 21/510, d1=0.000, d2=0.548 g=0.917
Epoch>1, 22/510, d1=0.002, d2=0.495 g=1.008
Epoch>1, 23/510, d1=0.004, d2=0.466 g=1.1